In [1]:
import requests
import json
import pandas as pd
from time import sleep

In [2]:
df = pd.read_csv('../output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Umbria
df_er = df_comuni[df_comuni['cod_regione'] == '10']

In [3]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

/tmp/ipykernel_1744/3249733948.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url_aff'] = df_er.apply(
/tmp/ipykernel_1744/3249733948.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url'] = df_er.apply(


,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune,url_aff,url
422,103040800100,CASTEL VISCARDO,CM,N,NaN,7,10,UMBRIA,304,TERNI,080,0100,https://eleapi.interno.gov.it/siel/PX/getprefe...,https://elezioni.interno.gov.it/risultati/2024...
380,103040580290,MONTE CASTELLO DI VIBIO,CM,N,NaN,7,10,UMBRIA,304,TERNI,058,0290,https://eleapi.interno.gov.it/siel/PX/getprefe...,https://elezioni.interno.gov.it/risultati/2024...
432,103040800200,MONTEGABBIONE,CM,N,NaN,7,10,UMBRIA,304,TERNI,080,0200,https://eleapi.interno.gov.it/siel/PX/getprefe...,https://elezioni.interno.gov.it/risultati/2024...


In [4]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [5]:
comuni_data = []

for idx, url in enumerate(df_er['url_aff']):
    print(f"\nProcessing record {idx + 1}")
    print(f"URL: {url}")

    response = requests.get(url, headers=headers)
    print(f"Response status code: {response.status_code}")
    
    data = response.json()

    comune = df_er.iloc[idx]['desc']
    provincia = df_er.iloc[idx]['provincia']
    regione = df_er.iloc[idx]['regione']
    url = df_er.iloc[idx]['url']

    print(f"Processing data for Comune: {comune}, Provincia: {provincia}")

    # Start with basic comune info
    comune_info = {
        'comune': comune,
        'provincia': provincia,
        'regione': regione,
        'url': url
    }
    
    # Group candidate votes by lista
    lista_votes = {}
    lista_names = {}

    # First, collect lista names and prepare vote tracking
    if 'liste' in data:
        for lista in data['liste']:
            lista_votes[lista['cod']] = 0
            lista_names[lista['cod']] = lista['desc']

    # Aggregate candidate votes by lista
    if 'cand' in data:
        for cand in data['cand']:
            lista_code = cand['cod_lis']
            lista_votes[lista_code] += cand['voti']

    # Add lista votes to comune_info
    for lista_code, votes in lista_votes.items():
        lista_name = lista_names.get(lista_code, f"Lista {lista_code}")
        comune_info[lista_name] = votes
        print(f"{lista_name}: {votes} votes")

    sezioni_scrutinate = data['liste'][0]['sez_perv']
    sezioni_tot = data['liste'][0]['sez_tot']

    # Optional: Verify consistency across lists
    if not all(lista.get('sez_perv') == sezioni_scrutinate for lista in data['liste']):
        print(f"Warning: Inconsistent scrutinized sections for {comune}")

    # Add to comune_info
    comune_info['sezioni_scrutinate'] = sezioni_scrutinate
    comune_info['sezioni_tot'] = sezioni_tot

    comuni_data.append(comune_info)
    
    print(f"Processed {comune}")
    sleep(0.1)

# Create DataFrame
print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nSample of processed data:")
print(df_data.head())


Processing record 1
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/10/PR/058/CM/0010


Response status code: 200
Processing data for Comune: ASSISI, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
Processed ASSISI

Processing record 2
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/10/PR/058/CM/0020


Response status code: 200
Processing data for Comune: BASTIA UMBRA, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 2 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
Processed BASTIA UMBRA

Processing record 3
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/10/PR/058/CM/0030


Response status code: 200
Processing data for Comune: BETTONA, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
Processed BETTONA

Processing record 4
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/10/PR/058/CM/0040


Response status code: 200
Processing data for Comune: BEVAGNA, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
Processed BEVAGNA

Processing record 5
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/10/PR/058/CM/0050


...

Response status code: 200
Processing data for Comune: PORANO, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
Processed PORANO

Processing record 89
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/10/PR/080/CM/0290


Response status code: 200
Processing data for Comune: SAN GEMINI, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
Processed SAN GEMINI

Processing record 90
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/10/PR/080/CM/0300


Response status code: 200
Processing data for Comune: SAN VENANZO, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 23 votes
ALTERNATIVA POPOLARE: 6 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 4 votes
PARTITO DEMOCRATICO: 44 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 5 votes
TESEI PRESIDENTE: 10 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 4 votes
ALLEANZA VERDI E SINISTRA: 24 votes
CIVICI UMBRI: 1 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 3 votes
Processed SAN VENANZO

Processing record 91
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/10/PR/080/CM/03

Response status code: 200
Processing data for Comune: STRONCONE, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
Processed STRONCONE

Processing record 92
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/10/PR/080/CM/0320


Response status code: 200
Processing data for Comune: TERNI, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
Processed TERNI

Creating DataFrame...

Sample of processed data:
                  comune provincia regione  \
0                 ASSISI     TE

In [6]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data

,comune,provincia,regione,url,FRONTE DEL DISSENSO,QUINTO POLO PER L'ITALIA,INSIEME PER UN'UMBRIA RESISTENTE,PIU' ITALIA SOVRANA,LEGA,ALTERNATIVA POPOLARE,...,ALLEANZA VERDI E SINISTRA,CIVICI UMBRI,ALTERNATIVA PER L'UMBRIA,FORZA DEL POPOLO,ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE,TRITTO PRESIDENTE UMANI INSIEME LIBERI,UMBRIA DOMANI PROIETTI PRESIDENTE,UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE,sezioni_scrutinate,sezioni_tot
0,Assisi,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,32
1,Bastia Umbra,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,22
2,Bettona,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,Bevagna,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
4,Campello Sul Clitunno,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Porano,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
88,San Gemini,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
89,San Venanzo,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,23,6,...,24,1,0,0,0,0,0,3,3,7
90,Stroncone,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6


In [7]:
df_data.to_csv('../output/risultati_um.csv', index=False, encoding='UTF-8-sig')